In [ ]:
import pandas as pd
import numpy as np
import os
import importlib

pd.set_option('display.max_columns', 50)

## Preliminaries

Run the `create_data.py` script to generate the following files:

- `cohort.csv`:
  - Contains one record for each adult patient’s first ICU stay over 48 hours in lengthwithin their first hospital admission.
  - The `mort_icu` column represents whether the patient died during their ICU stay.
  - The columns from `Acute Renal` to `Shock` correspond to each of the 25 CCS code  groups, which are derived from  ICD-9 codes assigned at the end of a patient’s hospital stay.
  - The `Any Acute` and `Any Chronic` columns are derived from whether the patient has any acute and chronic phenotypes respectively.
  
  
- `notes.csv`
  - Contains, for each of the patients in `cohort.csv`, all of the notes written during their hospital stay (along with the timestamp) for the following note types:
      - Discharge Summary
      - Nursing
      - Nursing/other
  - The notes have been lightly preprocessed (ex: removing PHI identifiers, removing section numbers).

In [ ]:
data_path = '/h/haoran/projects/hw2/mimic_data/'

In [ ]:
cohort = pd.read_hdf(os.path.join(data_path, 'cohort.h5'))
notes = pd.read_hdf(os.path.join(data_path, 'notes.h5'))

In [ ]:
cohort.head()

In [ ]:
cohort.shape

In [ ]:
notes.head()

In [ ]:
notes.shape

## Data Exploration

In [ ]:
## Write code for question 2 here

## Fairness Metrics

In [ ]:
%%html
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;width:100%}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-fymr{font-weight:bold;border-color:inherit;text-align:left;vertical-align:top}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-fymr" style="width:20%">Fairness Property</th>
    <th class="tg-fymr" style="width:40%">Definition</th>
    <th class="tg-fymr" style="width:15%">Gap Name</th>
    <th class="tg-fymr" style="width:25%">Gap Equation</th>
  </tr>
  <tr>
    <td class="tg-0pky">Demographic parity</td>
    <td class="tg-0pky">$P(\hat{Y}=y) = P(\hat{Y} = \hat{y} | Z=z),&nbsp;&nbsp;\forall z \in Z $</td>
    <td class="tg-0pky">Parity Gap</td>
    <td class="tg-0pky">$\frac{TP_1 + FP_1}{N_1} - \frac{TP_2 + FP_2}{N_2}$</td>
  </tr>
  <tr>
    <td class="tg-0pky">Equality of opportunity<br>(positive class)</td>
    <td class="tg-0pky">$P(\hat{Y}=1 | Y=1) =&nbsp;&nbsp;P(\hat{Y}=1 | Y=1, Z=z), \forall z \in Z$</td>
    <td class="tg-0pky">Recall Gap</td>
    <td class="tg-0pky">$\frac{TP_1 }{TP_1+FN_1} - \frac{TP_2 }{TP_2 + FN_2}$</td>
  </tr>
  <tr>
    <td class="tg-0pky">Equality of opportunity<br>(negative class)</td>
    <td class="tg-0pky">$P(\hat{Y}=0 | Y=0) =&nbsp;&nbsp;P(\hat{Y}=0 | Y=0, Z=z), \forall z \in Z$</td>
    <td class="tg-0pky">Specificity Gap</td>
    <td class="tg-0pky">$\frac{TN_1 }{TN_1+FP_1} - \frac{TN_2 }{TN_2 + FP_2}$</td>
  </tr>
</table>

In [ ]:
import metrics
importlib.reload(metrics)

def binary_test():
    test_df1 = pd.DataFrame(data  = {
        'protected': ['A','B']*5,
        'target': [0]*3 + [1]*7,
        'pred':[0.1, 0.8, 0.8, 0.9, 0.3, 0.3, 0.99, 0.1, 0.1, 0.9]
    })
    ret = metrics.gap_metrics(test_df1, 'protected', 'target', 'pred', 0.4)
    assert(np.allclose(ret['parity']['A'], -0.2) and
    np.allclose(ret['parity']['B'], 0.2) and
    np.allclose(ret['specificity']['A'], 0.5) and
    np.allclose(ret['specificity']['B'], -0.5) and
    np.allclose(ret['recall']['A'], -1/6) and
    np.allclose(ret['recall']['B'], 1/6)) , 'Test failed!'
    print("Test passed!")
    
binary_test()

## TF-IDF Model for ICU Mortality

In [ ]:
def process_data(cohort, notes):
    '''
    Processes data according to the steps in 4a
    Inputs :
        - cohort dataframe
        - notes dataframe
    Output:
        - Dataframe with (at least) the following columns:
            - subject_id, hadm_id
            - text
            - gender
            - mort_icu
            - train
    '''
    
    pass

df = process_data(cohort, notes)

In [ ]:
# code for 4a, 4b, 4c here


In [ ]:
#4d
train_df = df[df.train == 1] # you may want to create a validation set from this
test_df = df[df.train == 0] 

In [ ]:
# write your model training and evaluation code here


In [ ]:
# use this function in your solution for 4e, stratify by gender
def stratified_sample(df, column):
    '''
    Returns a stratified sample of df with replacement (stratified by column), with the same number of rows as df.
    '''
    grp = df.groupby(column, group_keys = False)
    return grp.apply(lambda x: x.sample(n = int(np.rint(len(x))), replace = True)).sample(frac=1).reset_index(drop = True)

In [ ]:
# Write code for 4e here
import metrics
importlib.reload(metrics)

